# Connect-4 AI Agent

This notebook implements Connect-4 and then uses minimax + a custom position evaluation algorithm to create an AI Connect-4 opponent

### Imports

In [13]:
import numpy as np
import copy
import operator
import random

### Define Game object

In [14]:
class Game:
    
    def __init__(self, size, from_state=None):
        self.size = size
        self.gameboard = np.zeros(shape=size)
        if from_state is not None:
            self.gameboard = from_state
        self.turn = 1
        
    def isLegal(self, col):
        '''
        Return if placing a piece in col is legal or not
        '''
        return (self.gameboard[0][col] == 0)
        
    def makeMove(self, col):
        '''
        Place piece in column
        '''
        if self.isLegal(col):
            #find first open row
            row = self.size[0]-1
            for i in range(self.size[0]):
                if self.gameboard[i][col] != 0:
                    row = i-1
                    break
                    
            self.gameboard[row][col] = self.turn
            self.turn += 1
            if self.turn > 2:
                self.turn = 1
                
    def isGameOver(self):
        '''
        Check if a player has won
        '''
        if len(self.legalMoves()) == 0:
            return 0
        
        #check rows
        for i in range(self.size[0]):
            row = "".join(str(self.gameboard[i,:].astype(np.int64))).replace(" ","").replace("[","").replace("]","")
            if row.count("1111") == 1:
                return 1
            if row.count("2222") == 1:
                return 2
        
        #check cols
        for i in range(self.size[1]):
            col = "".join(str(self.gameboard[:,i].astype(np.int64))).replace(" ","").replace("[","").replace("]","")
            
            if col.count("1111") == 1:
                return 1
            if col.count("2222") == 1:
                return 2
            
        #diagonals
        for x in range(self.size[0] - 3):
            for y in range(3, self.size[1]):
                for player in [1,2]:
                    if self.gameboard[x][y] == player and self.gameboard[x+1][y-1] == player and self.gameboard[x+2][y-2] == player and self.gameboard[x+3][y-3] == player:
                        return player

        for x in range(self.size[0] - 3):
            for y in range(self.size[1] - 3):
                for player in [1,2]:
                    if self.gameboard[x][y] == player and self.gameboard[x+1][y+1] == player and self.gameboard[x+2][y+2] == player and self.gameboard[x+3][y+3] == player:
                        return player
        
        return None
    
    def checkDiag(self, a):
        for i in range(-self.size[0]+1,self.size[0]):
            d = np.diagonal(a,i)
            d = "".join(str(d.astype(np.int64))).replace(" ","").replace("[","").replace("]","")
            if d.count("1111") == 1:
                return 1
            if d.count("2222") == 1:
                return 2
                    
        return None

    def legalMoves(self):
        return [i for i in range(self.size[0]) if self.isLegal(i)]
    

In [15]:
x = Game((6,7))
x.gameboard

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

### Define AI Agent object

In [16]:
class HardCode:
    def __init__(self, player, depth=3):
        
        self.player = player
        if self.player == 1:
            self.enemy = 2
        else:
            self.enemy = 1
        self.eval_matrix = np.zeros(shape=(6,7))
        
        for row in range(6):
            for col in range(7):
                
                value = 0
                
                h = abs(3-col)
                v = abs(3-row)
                
                value+=(4-h)
                value+=(4-v)
                
                value += self.diag_helper(row, col)
                
                self.eval_matrix[row][col] = value+1 +random.uniform(-2,2)
        self.depth = depth
        
    def diag_helper(self, row, col):
        '''
        Helper function that returns # of possible diagonal connect 4s at a specific row and column
        '''
        value = 0
        tmp_r = row
        tmp_c = col
        while tmp_r > 0 and tmp_c > 0:
            tmp_r -= 1
            tmp_c -= 1
        while tmp_r < 6 and tmp_c < 7:
            tmp_r += 1
            tmp_c += 1
            value += 1
        tmp_r = row
        tmp_c = col
        while tmp_r >= 0 and tmp_c < 7:
            tmp_r -= 1
            tmp_c += 1
        while tmp_r < 6 and tmp_c >= 0:
            tmp_r += 1
            tmp_c -= 1
            value += 1
            
        return value
                
    def evaluate(self, gameBoard):
        '''
        Return evaluation of a given gameboard
        '''
        
        b = gameBoard.gameboard
        if gameBoard.isGameOver() == self.enemy:
            return -100000
        elif gameBoard.isGameOver() == self.player:
            return 100000
        s = 0
        for row in range(6):
            for col in range(7):
                if b[row][col] == self.player:
                    s+=self.eval_matrix[row][col]
                if b[row][col] == self.enemy:
                    s-=self.eval_matrix[row][col]
        return s
    
    def choose_move(self, g, legal_moves, depth=None):
        '''
        Given a gameboard and set of legal moves, return the best move
        '''
        
        if depth==None:
            depth = self.depth
        tree = self.build_tree(g, depth=depth)
        tree = self.prune(tree)
        if type(tree) == int:
            return legal_moves[tree]
        if tree == None:
            return random.choice(legal_moves)
        subnodes = tree.subnodes
        #2nd to last layer
        d = 1
        while d < (2*depth):
            new = []
            for node in subnodes:
                if node is not None:
                    new.extend(node.subnodes)
            subnodes = new
            d+=1
        max_min = -np.Inf
        best_node = -1
        for i,node in enumerate(subnodes):
            if node is not None:
                local_min = np.Inf
                for j,child in enumerate(node.subnodes):
                    if child is not None:
                        score = self.evaluate(child.board)
                        if score < local_min:
                            local_min = score
                if local_min != np.Inf and local_min > max_min:
                    max_min=local_min
                    best_node = i
                    
        n = subnodes[best_node]
        while n.parent.root == False:
            n = n.parent
        best_node = n.index
        
        return legal_moves[best_node]
        
    
    def build_tree(self, game, depth):
        '''
        Build and return tree of future moves 
        '''

        root_node = Node(game, 0, -1, root=True)
        current_level = [root_node]
        for z in range(2*depth+1):
            new_level = []
            for node in current_level:
                
                poss_moves = node.board.legalMoves()
                new_gbs = [copy.deepcopy(node.board) for move in poss_moves]
                for i in range(len(new_gbs)):
                    new_gbs[i].makeMove(poss_moves[i])
                new_nodes = [Node(new_gbs[i], z+1, i) for i in range(len(new_gbs))]
                for i,child in enumerate(new_nodes):
                    new_level.append(child)
                    node.set_child(child, poss_moves[i])
            current_level = new_level
        return root_node
        
    
    def prune(self, t):
        '''
        Manually remove branches of tree which lead to game loss 
        '''
        
        for i,child in enumerate(t.subnodes):
            if child is not None:
                score = self.evaluate(child.board)
                if score == 100000:
                    return child.index
                for child2 in child.subnodes:
                    if child2 is not None:
                        score2 = self.evaluate(child2.board)
                        if score2 == -100000:
                            t.subnodes[i] = None
                            break
                        for child3 in child2.subnodes:
                            if child3 is not None:
                                bad_children = 0
                                for child4 in child3.subnodes:
                                    if child4 is not None:
                                        score3 = self.evaluate(child4.board)
                                        if score3 == -100000:
                                            bad_children += 1
                                        if bad_children == 2:
                                            t.subnodes[i] = None
                                            break
        for sub in t.subnodes:
            if sub is not None:
                return t
        return None
            

### Define Node object to build trees

In [17]:
class Node:
    def __init__(self, board, depth, index, root=False):
        self.board = board
        self.subnodes = np.array([None,None,None,None,None,None,None])
        self.parent = None
        self.root = root
        self.depth = depth
        self.index = index
        
    def set_child(self, node, i):
        self.subnodes[i] = node
        node.set_parent(self)
        
    def set_parent(self, node):
        self.parent = node
        

### Define View and Controller objects for Connect-4 implementation

In [18]:
class View:
    def __init__(self):
        pass
    def drawboard(self, board):
        print(board)
        print("\n\n\n")

In [19]:
class Controller:
    def __init__(self, shape, players):
        self.shape = shape
        self.players = players
        self.view = View()
        
    def playGame(self, display=False):
        local = Game(self.shape)
        while not local.isGameOver():
            
            turn = local.turn
            player = self.players[turn-1]
            move = player.choose_move(local, local.legalMoves())
            
            local.makeMove(move)
            if display:
                self.view.drawboard(local.gameboard)
            
        return local.isGameOver()

In [20]:
class User:
    
    def __init__(self):
        pass
    
    def choose_move(self, a, b):
        move = int(input("Choose move:"))
        return move

In [21]:
class Random:
    def __init__(self):
        pass
    
    def choose_move(self, a, moves):
        return random.choice(moves)

### Play against the AI!

In [26]:
shape = (6,7)
p1 = User()
p2 = HardCode(2, depth=2)
j = Controller(shape, [p1, p2])

In [ ]:
j.playGame(display=True)

###### 